In [ ]:
# Reference: https://towardsdatascience.com/web-scraping-job-postings-from-indeed-com-using-selenium-5ae58d155daf

# Idea -> Use both appraoches
# Crawl general information with appraoch from Scrape_01
# And access hyperlinks with selenium.webdriver!

# With this way, we can filter out job titles, that we are not interested in and focus on the relevant ones!
# i.e. filter out "Praktikum", "Data Entry", etc.



In [8]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [36]:
import urllib
import time
import pandas as pd

In [5]:
# List of drivers for different web browsers
# https://selenium-python.readthedocs.io/installation.html


# !!! I downloaded the windows driver! !!!


# Specify path to firefox driver
DRIVER_PATH = '../Drivers/geckodriver_windows'
driver = webdriver.Firefox(executable_path = DRIVER_PATH)

In [28]:
# Navigate to indeed country webpage with prespecified query values
# Do not waste time with clicking and advanced searches.
indeed_country_url = "https://de.indeed.com/jobs?"
job_title = "Data Analyst"
location = "Berlin"
n_page = 1

getVars = {'q' : job_title, 'l' : location, 'fromage' : 'last', 'sort' : 'date'} #, 'start' : (n_page - 1) * 10
url = (indeed_country_url + urllib.parse.urlencode(getVars))
print(url)
driver.get(url)

https://de.indeed.com/jobs?q=Data+Analyst&l=Berlin&fromage=last&sort=date


In [ ]:
# Take care of popups!

# Cookie banner!

# Newsletter!
# close_popup = driver.find_element_by_id("popover-close-link")
# close_popup.click()

In [31]:
# Get information from jobcards

driver.implicitly_wait(3)

titles=[]
companies=[]
locations=[]
links =[]
reviews=[]
salaries = []



for i in range(0,3):

    print("Page: {}".format(str(i+1)))
    
    job_card = driver.find_elements_by_xpath('//div[contains(@class,"clickcard")]')
    
    for job in job_card:
       
    #.  not all companies have review
        try:
            review = job.find_element_by_xpath('.//span[@class="ratingsContent"]').text
        except:
            review = "None"
        reviews.append(review)
   #.   not all positions have salary
        try:
            salary = job.find_element_by_xpath('.//span[@class="salaryText"]').text
        except:
            salary = "None"
    #.  tells only to look at the element       
        salaries.append(salary)
        
        try:
            location = job.find_element_by_xpath('.//span[contains(@class,"location")]').text
        except:
            location = "None"
    #.  tells only to look at the element       
        locations.append(location)
        
        try:
            title  = job.find_element_by_xpath('.//h2[@class="title"]//a').text
        except:
            title = job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="title")
        titles.append(title)
        links.append(job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="href"))
        companies.append(job.find_element_by_xpath('.//span[@class="company"]').text)
        
    # Do not click for next page, just load it! :D
    url_loop = url + "&start=" + str((i + 1) * 10)
    print(url_loop)
    try:
        driver.get(url_loop)
    except:
        break

    time.sleep(0.5) # Do not overshoot!


    # try:
    #     next_page = driver.find_element_by_xpath('//a[@aria-label={}]//span[@class="pn"]'.format(i+2))
    #     next_page.click()

    # except:

    #     try:
    #         next_page = driver.find_element_by_xpath('//a[@aria-label="Next"]//span[@class="np"]')
    #         next_page.click()
    #     except:
    #         next_page = driver.find_element_by_xpath('//a[@aria-label="Weiter"]//span[@class="pn"]')
    #         next_page.click()

    # Depending on language of indeed website, there can be "weiter" or a random order of np / pn classes.

    #except:
        #next_page = driver.find_element_by_xpath('//a[.//span[contains(text(),"Next")]]')
        #next_page.click()
        
    
    



Page: 1
https://de.indeed.com/jobs?q=Data+Analyst&l=Berlin&fromage=last&sort=date&start=10
Page: 2
https://de.indeed.com/jobs?q=Data+Analyst&l=Berlin&fromage=last&sort=date&start=20
Page: 3
https://de.indeed.com/jobs?q=Data+Analyst&l=Berlin&fromage=last&sort=date&start=30


In [34]:
# Get job descriptions from each link
descriptions=[]
for link in links:
    print(link)
    driver.get(link)
    driver.implicitly_wait(0.5)
    jd = driver.find_element_by_xpath('//div[@id="jobDescriptionText"]').text
    descriptions.append(jd)
    time.sleep(1) # Slow down!


https://de.indeed.com/rc/clk?jk=984bfe5affaf05b6&fccid=40aeec9ab4c6bacc&vjs=3
https://de.indeed.com/rc/clk?jk=654c02551a76c0cd&fccid=9f3083d57395942b&vjs=3
https://de.indeed.com/rc/clk?jk=5234276f444e99ca&fccid=09704f60e923fb83&vjs=3
https://de.indeed.com/rc/clk?jk=af2ab965f6fe44ab&fccid=04ef9247542a6c50&vjs=3
https://de.indeed.com/rc/clk?jk=84c160c0e9bd168a&fccid=9514e4bb9d86329e&vjs=3
https://de.indeed.com/rc/clk?jk=b75d8156937efd38&fccid=18ff0230d75e9a90&vjs=3
https://de.indeed.com/rc/clk?jk=45503cb8b7054075&fccid=d519a1d6f9bfd5e2&vjs=3
https://de.indeed.com/rc/clk?jk=ba680a4dcdf5b611&fccid=9f3083d57395942b&vjs=3
https://de.indeed.com/rc/clk?jk=bdc5ebb19134b905&fccid=6fa5cbc87f52e4e1&vjs=3
https://de.indeed.com/rc/clk?jk=b357b3698882ceee&fccid=d1ba96e1329c2594&vjs=3
https://de.indeed.com/rc/clk?jk=7f39f2435aee5c79&fccid=35678f1e8c0a77cb&vjs=3
https://de.indeed.com/rc/clk?jk=17df081ab6d05a2b&fccid=3228946bc134183e&vjs=3
https://de.indeed.com/rc/clk?jk=5311bdf2a0ab44c2&fccid=10da2bb49

In [47]:
# descriptions = descriptions.pop(0)

In [48]:
df = pd.DataFrame()
df['Title'] = titles
df['Company'] = companies
df['Location'] = locations
df['Link'] = links
df['Review'] = reviews
df['Salary'] = salaries
df['Description'] = descriptions

In [50]:
print(df)

About Fraugster:
Fraugster’s purpose is to reduce the spiralling levels of online fraud, estimated to reach $1 Trillion by the end of 2021. Our mission is to protect companies selling online, and intelligently manage the impacts of fraud on their business and their customers. We do this by using advanced AI to spot patterns of fraudulent behaviour, and stop them. The combination of our expertise and technology makes us one of the most accurate solutions in the market, and our ambition is to set a new standard of online trust, globally. Our customers are some of the worlds largest payment companies and most renowned brands in e-commerce, and we have the backing of Europe's premier tech funds (Speedinvest, CommerzVentures, Munich Re Ventures and Earlybird). We are a very diverse bunch of technologists and problem solvers from over 25 countries across the globe. Our culture is defined by intellectual curiosity, teamwork and a passion for solving complex problems. If this sounds like an en